<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:250%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת שכבת חלקות (דו מימדיות) לבנק"ל מודרני 🌐

In [ ]:
print('11. Parcels borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  11.a Reading inputs')

In [ ]:
RawData_Folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

today =  dt.date.today()
product_name =  str(today.year) + str(today.month) + str(today.day)  + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + product_name

del [today, product_name]

<h1 style="text-align:right;font-size:125%">
נתוני חלקות
<h1 style="text-align:right;font-size:100%">
חלקות פעילות

In [ ]:
parcel_all_cols = ['Identifier', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'STATUS', 'LEGAL_AREA', 'TALAR_NUMBER', 'TALAR_YEAR', 'geometry']

parcel_all = gpd.read_file(RawData_Folder + 'ArcStorm\PARCEL_ALL.gdb', 
                           layer = 'PARCEL_ALL', 
                           include_fields = parcel_all_cols)

parcel_all = parcel_all.to_crs(2039)

del parcel_all_cols

print('         input 1/9 read')

<h1 style="text-align:right;font-size:100%">
חלקות בתהליך

In [ ]:
parcels_inprocess_cols = ['GUSHNUM', 'GUSHSUFFIX', 'PARCELNAME', 'TALAR_NUM', 'TALAR_YEAR', 'CADASTER_PROCESS']

parcels_inprocess = gpd.read_file(RawData_Folder + 'CADSDE\Parcels_InProcess.gdb',
                                  include_fields = parcels_inprocess_cols,
                                  ignore_geometry = True)

del parcels_inprocess_cols

print('         input 2/9 read')

<h1 style="text-align:right;font-size:125%">
    נתוני גושים מודרניים

In [ ]:
blocks_cols = ['BlockUniqueID', 'BlockNumber', 'SubBlockNumber']
blocks = gpd.read_file(product_path, layer = 'Blocks',
                       include_fields = blocks_cols,
                       ignore_geometry = True)

del blocks_cols

print('         input 3/9 read')

<h1 style="text-align:right;font-size:125%">
נתוני תהליכי קדסטר מודרניים ופיצול לפי סוג תהליך
<h1 style="text-align:right;font-size:100%">
תצ"ר
<h1 style="text-align:right;font-size:100%">
תת"ג
<h1 style="text-align:right;font-size:100%">
פס"ד    
<h1 style="text-align:right;font-size:100%">
תיקוני 97 ב

In [ ]:
cpb_columns = ['CPBUniqueID', 'BlockUniqueID', 'ProcessType', 'ProcessName']

cpb_layer = gpd.read_file(product_path,
                          layer = 'CadasterProcessBorders',
                          include_fields = cpb_columns,
                          ignore_geometry = True)

cpb_layer['BlockUniqueID'].fillna(0, inplace=True)
cpb_layer[['CPBUniqueID', 'BlockUniqueID','ProcessType']] = cpb_layer[['CPBUniqueID', 'BlockUniqueID','ProcessType']].astype(int)


tazar_layer = cpb_layer[cpb_layer['ProcessType'] == 1]

tatag_layer = cpb_layer[cpb_layer['ProcessType'] == 4]

judgments_layer = cpb_layer[cpb_layer['ProcessType'] == 3]

amendments97b_layer = cpb_layer[cpb_layer['ProcessType'] == 6]


del [cpb_columns, product_path]

print('         input 4/9 read')

<h1 style="text-align:right;font-size:125%">
נתוני תהליכי קדסטר טבלאיים
<h1 style="text-align:right;font-size:100%">
חלקות פס"ד וטבלת פס"ד

In [ ]:
judgments_parcels_cols = ['PSAK_DIN_ID', 'PARCEL']
judgments_parcels = pd.read_csv(RawData_Folder + 'Oracle\psak_din_parcels.csv',
                                usecols = judgments_parcels_cols)

print('         input 5/9 read')

judgments_cols = ['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'GUSH_NUM', 'GUSH_SUFFIX']
judgments = pd.read_csv(RawData_Folder + 'Oracle\psak_din.csv',
                        usecols = judgments_cols,
                        encoding = 'cp1255')

judgments_parcels = judgments_parcels.merge(right = judgments,
                                            how = 'left',
                                            on = 'PSAK_DIN_ID',
                                            validate = 'many_to_many')

del [judgments_parcels_cols, judgments_cols, judgments]

print('         input 6/9 read')

<h1 style="text-align:right;font-size:100%">
חלקות תיקוני 97 ב

In [ ]:
amendments97b_cols = ['גוש','תת גוש','חלקה','מספר פניה']
amendments97b = pd.read_csv(RawData_Folder + 'Local\97b_nuhil_tahalichim.csv',
                            encoding = 'cp1255',
                            usecols = amendments97b_cols)
del amendments97b_cols

print('         input 7/9 read')

<h1 style="text-align:right;font-size:100%">
    טבלת תצ"ר מסוג כרטסת

In [ ]:
tazar_card_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'TALAR_ID', 'FINAL_PARCEL']
tazar_card = pd.read_csv(RawData_Folder + "Oracle/talar_card_dest.csv",
                         usecols = tazar_card_cols)
del tazar_card_cols

print('         input 8/9 read')

<h1 style="text-align:right;font-size:100%">
    מידע לסטאטוסים של גבולות תת"ג קיימים

In [ ]:
approved_tatag_dates_columns = ['TATAG_NUM', 'TATAG_YEAR', 'TATAG_STATUS', 'TATAG_STATUS_DATE']
approved_tatag_dates = gpd.read_file(RawData_Folder + 'CADSDE/TATAG.gdb',
                                     layer = 'TATAG',
                                     include_fields = approved_tatag_dates_columns,
                                     ignore_geometry = True)
del approved_tatag_dates_columns

print('         input 9/9 read')

<h1 style="text-align:right;font-size:100%">
נתוני מקרקעי ייעוד של חלקות מלשכת רישום

In [ ]:
SubBlocksList = pd.read_csv(r"\\mapi_shares\MNCDB\צוות מידע\RawData\Local\SubBlocksList.csv")
xml_df = pd.read_csv(r"\\mapi_shares\MNCDB\צוות מידע\RawData\Local\KeshetRimonAssets.csv").sort_values(['BlockName', 'ParcelNumber'])

xml_df = xml_df.merge(SubBlocksList[['BlockNumber', 'SubBlockNumber', 'BlockName']], on = 'BlockName', how = 'left')

del SubBlocksList

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

<h1 style="text-align:right;font-size:100%">
עיבוד נתוני מקרקעי ייעוד לפי שם חלקה מלא

In [ ]:
xml_df['BlockNumber'] = xml_df['BlockNumber'].fillna(xml_df['BlockName'])

xml_df['SubBlockNumber'] = xml_df['SubBlockNumber'].fillna(0)

xml_df = xml_df.drop(columns = 'BlockName')

xml_df[['BlockNumber', 'SubBlockNumber', 'ParcelNumber']] = xml_df[['BlockNumber', 'SubBlockNumber', 'ParcelNumber']].astype(int)

xml_df['ParcelName'] = xml_df['BlockNumber'].astype(str) + '_' +  xml_df['SubBlockNumber'].astype(str) + '_' + xml_df['ParcelNumber'].astype(str)

xml_df = xml_df[['ParcelName', 'LandDesignations']]

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני חלקות פעילות

<h1 style="text-align:right;font-size:100%">
סינון חלקות לפי סטאטוס של חלקות מוסדרות
<h1 style="text-align:right;font-size:100%">
סינון חלקות שמספרן או מספר הגוש שלהן הוא 0 או ריק
<h1 style="text-align:right;font-size:100%">
יצירת שדות מקשרים להמשך: שם חלקה מלא ושם גוש מלא
<h1 style="text-align:right;font-size:100%">
מיזוג נתוני חלקות עם נתוני גוש
<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות לפי אפיון חדש
<h1 style="text-align:right;font-size:100%">
איחוד עם נתוני מקרקעי יעוד

In [ ]:
print('  11.b Cleaning data')

In [ ]:
parcel_all = parcel_all[parcel_all['STATUS'].isin([1, 3, 6, 16])]
parcel_all.drop(columns = 'STATUS', inplace=True)

In [ ]:
parcel_all = parcel_all[(~parcel_all['PARCEL'].isna()) | (parcel_all['PARCEL'] != 0)]
parcel_all = parcel_all[(~parcel_all['GUSH_NUM'].isna()) | (parcel_all['GUSH_NUM'] != 0)]

In [ ]:
blocks = blocks.astype(int)

parcel_all[['GUSH_SUFFIX', 'TALAR_NUMBER', 'TALAR_YEAR']] = parcel_all[['GUSH_SUFFIX', 'TALAR_NUMBER', 'TALAR_YEAR']].fillna(0)
parcel_all[['GUSH_NUM', 'GUSH_SUFFIX', 'TALAR_NUMBER', 'TALAR_YEAR', 'PARCEL']] = parcel_all[['GUSH_NUM', 'GUSH_SUFFIX', 'TALAR_NUMBER', 'TALAR_YEAR', 'PARCEL']].astype(int)

blocks['BlockName'] = blocks['BlockNumber'].astype(str) + '_' + blocks['SubBlockNumber'].astype(str)
parcel_all['BlockName'] = parcel_all['GUSH_NUM'].astype(str) + '_' + parcel_all['GUSH_SUFFIX'].astype(str)
parcel_all.drop(columns=['GUSH_NUM', 'GUSH_SUFFIX'], inplace=True)


parcel_all['ParcelName'] = parcel_all['BlockName'] + '_' + parcel_all['PARCEL'].astype(str)

parcel_all['ProcessName'] = parcel_all['TALAR_NUMBER'].astype(str) + '/' + parcel_all['TALAR_YEAR'].astype(str)
parcel_all.drop(columns = ['TALAR_NUMBER', 'TALAR_YEAR'], inplace=True)
parcel_all['ProcessName'].replace('0/0', np.nan, inplace=True)

parcel_all = parcel_all.merge(right = blocks[['BlockUniqueID', 'BlockName']], on = 'BlockName', how = 'left')
parcel_all.dropna(subset=['BlockUniqueID'], inplace=True)
parcel_all[['Identifier', 'BlockUniqueID']] = parcel_all[['Identifier', 'BlockUniqueID']].astype(int)
parcel_all.rename(columns={'Identifier':'ParcelUniqueID', 'PARCEL':'ParcelNumber', 'LEGAL_AREA':'LegalArea'}, inplace=True)

del blocks

In [ ]:
parcel_all = parcel_all.merge(right = xml_df, on = 'ParcelName', how = 'left')

del xml_df

In [ ]:
parcel_x_process = parcel_all.merge(cpb_layer[['ProcessName', 'ProcessType', 'CPBUniqueID']], on = 'ProcessName', how = 'left')
parcel_with_pt = parcel_x_process[~parcel_x_process['ProcessType'].isna()]
parcel_without_pt = parcel_x_process[parcel_x_process['ProcessType'].isna()].drop(columns = ['CPBUniqueID', 'ProcessType', 'ProcessName'])

<h1 style="text-align:right;font-size:125%">
    
:עיבוד נתוני חלקות בתהליך

<h1 style="text-align:right;font-size:100%;">


    תיקון ערכים בשדות אליפסה/מלבן

    יצירת שדה של שם תכנית מלא
 
    תיקון ערכים בשדה מספר חלקה
 
    יצירת שדה של שם חלקה מלא

In [ ]:
parcels_inprocess[['TALAR_NUM', 'TALAR_YEAR']] = parcels_inprocess[['TALAR_NUM', 'TALAR_YEAR']].fillna(0)
parcels_inprocess[['TALAR_NUM', 'TALAR_YEAR']] = parcels_inprocess[['TALAR_NUM', 'TALAR_YEAR']].astype(int)
parcels_inprocess['ProcessName'] = parcels_inprocess['TALAR_NUM'].astype(str) + '/' + parcels_inprocess['TALAR_YEAR'].astype(str)
parcels_inprocess.drop(columns = ['TALAR_NUM', 'TALAR_YEAR'], inplace=True)


#Remove F,T characters
parcels_inprocess['PARCELNAME'] = parcels_inprocess['PARCELNAME'].map(lambda x: x.lstrip('FT'))
#Remove bad characters and keep parcel number only
parcels_inprocess['PARCELNAME'] = parcels_inprocess['PARCELNAME'].str.partition(":")[0]


parcels_inprocess['ParcelName'] = parcels_inprocess['PARCELNAME'].astype(int)
parcels_inprocess['ParcelName'] = parcels_inprocess['GUSHNUM'].astype(str) + '_' + parcels_inprocess['GUSHSUFFIX'].astype(str) +  '_' + parcels_inprocess['ParcelName'].astype(str)
parcels_inprocess.drop(columns=['PARCELNAME', 'GUSHNUM', 'GUSHSUFFIX'], inplace=True)

<h1 style="text-align:right;font-size:125%;">
 :עיבוד נתוני תצ"ר כרטסת

<h1 style="text-align:right;font-size:100%;">


    הסרת רשומות ריקות מערכים

    הגדרת כל השדות כמספר שלם
 
    יצירת שדה שם חלקה מלא
 
    סינון רק לחלקות שמופיעות בשכבת חלקות פעילהושאין להן תהליך יוצר
 
    יצירת שדה עדכני של מספר מזהה לתצ"ר
    
    הסרת שדות שאינם רלוונטיים
    
    סינון לחלקות אשר תואמות לפי שדה של מס' מזהה לתהליך בין טבלת כרטסת לשכבת גבולות תצ"ר מודרניים
    
    איכלוס השדה- סוג תהליך בערך 1

In [ ]:
print('  11.c Data conversion')

In [ ]:
tazar_card.dropna(inplace=True)
tazar_card = tazar_card.astype(int)

tazar_card['ParcelName'] = tazar_card['GUSH_NUM'].astype(str) + '_' + tazar_card['GUSH_SUFFIX'].astype(str) + '_' + tazar_card['FINAL_PARCEL'].astype(str)

tazar_card = tazar_card[tazar_card['ParcelName'].isin(parcel_without_pt['ParcelName'])]

tazar_card['CPBUniqueID'] = tazar_card['TALAR_ID'] + 1000000

tazar_card.drop(columns = ['GUSH_NUM', 'GUSH_SUFFIX', 'FINAL_PARCEL', 'TALAR_ID'], inplace = True)

tazar_card = tazar_card[tazar_card['CPBUniqueID'].isin(cpb_layer['CPBUniqueID'])]

tazar_card['ProcessType'] = 1 #CreateProcessLut

<h1 style="text-align:right;font-size:100%;">

    מיזוג טבלת כרטסת עם שכבת חלקות פעילה שללא תהליך יוצר

    הוספת האיחוד לשכבת חלקות פעילה (כי לאחר מיזוג יש תהליך יוצר)
 
    עדכון שכבת חלקות ללא תהליך יוצר

In [ ]:
tazar_card = parcel_without_pt.merge(tazar_card, how = 'inner', on = 'ParcelName')

parcel_with_pt = parcel_with_pt.append(tazar_card)

parcel_without_pt = parcel_without_pt[~parcel_without_pt['ParcelName'].isin(tazar_card['ParcelName'])]

del tazar_card

<h1 style="text-align:right;font-size:125%;">
    :עיבוד נתוני פסקי דין

<h1 style="text-align:right;font-size:100%;">

    הסרת רשומות בעלות מספר גוש חסר
    
    השלמת ערכים לשדה תת-גוש עם הערך 0
    
    הגדרת שדות מספריים למספריים שלמים
    
    הוספת שדה למספר חלקה מלא
    
    הוספת 40000 למספר פסק הדין
    
    הוספת שדה למספר תהליך קדסטרי
    
    הסרת שדות שאינם רלוונטיים יותר
    
    סינון לחלקות פס"ד שנמצאות בשכבת חלקות פעילה וללא תהליך יוצר
    
    איחוד חלקות פס"ד עם שכבת גבולות פס"ד מודרניים
    
    הגדרת שדה מספר ייחודי לתהליך כמספר שלם
    
    הסרת שדה לא רלוונטי
    
    איחוד התוצאה עם שכבת חלקות פעילה וללא תהליך יוצר
    
    איכלוס השדה סוג תהליך ב-2

In [ ]:
judgments_parcels.dropna(subset=['GUSH_NUM'], inplace=True)

judgments_parcels['GUSH_SUFFIX'] = judgments_parcels['GUSH_SUFFIX'].fillna(0)

judgments_parcels[['GUSH_SUFFIX', 'GUSH_NUM', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'PARCEL']] = judgments_parcels[['GUSH_SUFFIX', 'GUSH_NUM', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'PARCEL']].astype(int)

judgments_parcels['ParcelName'] = judgments_parcels['GUSH_NUM'].astype(str) + '_' + judgments_parcels['GUSH_SUFFIX'].astype(str) + '_' + judgments_parcels['PARCEL'].astype(str)

judgments_parcels['PSAK_DIN_NUM'] = judgments_parcels['PSAK_DIN_NUM'].astype(int)

judgments_parcels['ProcessName'] = judgments_parcels['PSAK_DIN_NUM'].astype(str) + '-' + judgments_parcels['PSAK_DIN_YEAR'].astype(str)

judgments_parcels.drop(columns=['PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL'], inplace=True)

judgments_parcels = judgments_parcels[judgments_parcels['ParcelName'].isin(parcel_without_pt['ParcelName'])]

judgments_parcels = judgments_parcels.merge(judgments_layer[['CPBUniqueID','ProcessName']] , how = 'inner', on = 'ProcessName')

judgments_parcels['CPBUniqueID'] = judgments_parcels['CPBUniqueID'].astype(int)
judgments_parcels.drop(columns='PSAK_DIN_ID', inplace=True)

judgments_parcels = parcel_without_pt.merge(judgments_parcels, how='inner', on='ParcelName')

judgments_parcels['ProcessType'] = 2  #CreateProcessLut

In [ ]:
parcel_with_pt = parcel_with_pt.append(judgments_parcels)
parcel_without_pt = parcel_without_pt[~parcel_without_pt['ParcelName'].isin(judgments_parcels['ParcelName'])]

In [ ]:
parcel_x_process = parcel_with_pt.append(parcel_without_pt)

parcel_x_process['ProcessType'].fillna(3, inplace=True) #CreateProcessLut

parcel_x_process[['ParcelUniqueID','ParcelNumber','ProcessType', 'CPBUniqueID']] = parcel_x_process[['ParcelUniqueID','ParcelNumber','ProcessType', 'CPBUniqueID']].astype('Int32')

parcel_x_process.rename(columns={'ProcessType':'CreateProcessType', 'CPBUniqueID' : 'CPBUniqueCreateProcessID'}, inplace=True)

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני תת"ג
    
<h1 style="text-align:right;font-size:100%;">
 בחירת חלקות בתהליך אשר מקורן בתת"ג
    <h1 style="text-align:right;font-size:100%;">
    הסרת שדה של סוד תהליך
        <h1 style="text-align:right;font-size:100%;">
    איחוד של חלקות תת"ג בתהליך עם שכבת גבולות תת"ג
            <h1 style="text-align:right;font-size:100%;">
    הסרת ערכים ריקים והגדרת שדות למספרים שלמים
                <h1 style="text-align:right;font-size:100%;">
    סינון של תוצאת המיזוג לחלקות אשר נמצאות בשכבת חלקות
    

In [ ]:
parcels_inprocess_tatag = parcels_inprocess[parcels_inprocess['CADASTER_PROCESS']==12]
parcels_inprocess_tatag.drop(columns = 'CADASTER_PROCESS', inplace=True)

del parcels_inprocess

In [ ]:
tatag_parcels = parcels_inprocess_tatag.merge(tatag_layer, how='left', on='ProcessName')
tatag_parcels.dropna(subset=['CPBUniqueID', 'BlockUniqueID'], inplace=True)
tatag_parcels[['CPBUniqueID', 'BlockUniqueID', 'ProcessType']] = tatag_parcels[['CPBUniqueID', 'BlockUniqueID', 'ProcessType']].astype('Int32')
del parcels_inprocess_tatag


tatag_parcels = tatag_parcels[tatag_parcels['ParcelName'].isin(parcel_x_process['ParcelName'])]
tatag_parcels.rename(columns={'CPBUniqueID' : 'CPBUniqueUpdateProcessID'}, inplace=True)
tatag_parcels.sort_values(['ParcelName', 'ProcessName'], inplace=True)

<h1 style="text-align:right;font-size:125%;">
:עיבוד טבלת סטאטוסים של תת"ג מאושרים
<h1 style="text-align:right;font-size:100%;">    
איחוד של טבלת סטאטוסים ותאריכים עם טבלת חלקות תת"ג בתהליך
<h1 style="text-align:right;font-size:100%;">    
מיון של הטבלה לפי שם חלקה ותאריך הסטאטוס שלה
<h1 style="text-align:right;font-size:100%;">    
הורדת כפילויות של שם חלקה ושמירת התאריך סטאטוס תת"ג האחרון ביותר

In [ ]:
approved_tatag_dates = approved_tatag_dates[approved_tatag_dates['TATAG_STATUS'] == 2]
approved_tatag_dates['ProcessName'] = approved_tatag_dates['TATAG_NUM'].astype(str) + '/' + approved_tatag_dates['TATAG_YEAR'].astype(str)
approved_tatag_dates['TATAG_STATUS_DATE'] = pd.to_datetime(approved_tatag_dates['TATAG_STATUS_DATE'])
approved_tatag_dates.drop(columns = ['TATAG_STATUS', 'TATAG_NUM', 'TATAG_YEAR'], inplace=True)


tatag_parcels = tatag_parcels.merge(approved_tatag_dates, on='ProcessName', how='left')
tatag_parcels['TATAG_STATUS_DATE'].fillna('1800-01-01', inplace=True)
tatag_parcels.sort_values(by=['ParcelName', 'TATAG_STATUS_DATE','ProcessName'], inplace=True)
tatag_parcels = tatag_parcels.drop_duplicates(subset=['ParcelName'], keep='first')

<h1 style="text-align:right;font-size:100%;">
מיזוג עם נתוני חלקות פעילות

In [ ]:
parcel_x_process = parcel_x_process.merge(tatag_parcels[['ParcelName','CPBUniqueUpdateProcessID']],
                                          on = 'ParcelName',
                                          how = 'left')

del [tatag_parcels, approved_tatag_dates]

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני תיקוני 97ב
    
<h1 style="text-align:right;font-size:100%;">
 הסרת רשומות של חלקות ללא מדיע של גוש או חלקה
<h1 style="text-align:right;font-size:100%;">
   מילוי ערכי תת-גוש חסרים ב-0
    <h1 style="text-align:right;font-size:100%;">
    יצירת שדה של שם חלקה מלא
    

In [ ]:
amendments97b.dropna(subset=['גוש', 'חלקה'], inplace=True)
amendments97b['תת גוש'].fillna(0, inplace=True)
amendments97b['תת גוש'] = amendments97b['תת גוש'].astype(int)
amendments97b['ProcessName'] = amendments97b['חלקה'].astype(str) + '/' + amendments97b['גוש'].astype(str) + '/' + amendments97b['תת גוש'].astype(str)
amendments97b['ParcelName'] = amendments97b['גוש'].astype(str) + '/' + amendments97b['תת גוש'].astype(str) + '/' + amendments97b['חלקה'].astype(str)

amendments97b.drop(columns=['גוש','תת גוש','חלקה'], inplace = True)
amendments97b['מספר פניה'] = amendments97b['מספר פניה'].astype('Int32')

<h1 style="text-align:right;font-size:100%;">
    מיזוג שכבת גבולות 97ב עם טבלת חלקות 97ב
    <h1 style="text-align:right;font-size:100%;">
        הסרת שדה לא רלוונטי
        <h1 style="text-align:right;font-size:100%;">
            הגדרת שדה מספר מזהה של תהליך 

In [ ]:
amendments97b = amendments97b.merge(amendments97b_layer[['CPBUniqueID', 'ProcessName']], how='left', on='ProcessName')
amendments97b['CPBUniqueID'] = 5000000 + amendments97b['מספר פניה']
amendments97b['CPBUniqueID'] = amendments97b['CPBUniqueID'].astype(int)

<h1 style="text-align:right;font-size:100%;">
    הסרת שדה לא רלוונטי
    <h1 style="text-align:right;font-size:100%;">
    מיזוג חלקות 97ב עם שכבת חלקות פעילה    

In [ ]:
parcel_x_process = parcel_x_process.merge(right = amendments97b[['CPBUniqueID', 'ParcelName']],
                                          on = 'ParcelName',
                                          how = 'left')

parcel_x_process['CPBUniqueUpdateProcessID'] = parcel_x_process['CPBUniqueUpdateProcessID'].fillna(parcel_x_process['CPBUniqueID'])

parcel_x_process = parcel_x_process.drop(columns = ['CPBUniqueID'])
del [amendments97b, amendments97b_layer]

<h1 style="text-align:right;font-size:125%;">
    :עריכת שכבת חלקות פעילה לאחר תוצאות

In [ ]:
print('  11.d Defining new Layer')

<h1 style="text-align:right;font-size:100%;">
הוספת שדה של מספר מזהה לתהליך מבטל
<h1 style="text-align:right;font-size:100%;">
הוספת שדה של סוג חלקה
<h1 style="text-align:right;font-size:100%;">
הוספת 5,000,000 למספר מזהה של חלקה
<h1 style="text-align:right;font-size:100%;">
תיקון סוגי שדות למספרים שלמים
<h1 style="text-align:right;font-size:100%;">
הסרת כפילויות (ישנן חלקות שהוכפלו במיזוג בגלל נמצאו להן יותר מתהליך מעדכן אחד)
<h1 style="text-align:right;font-size:100%;">
הסרת שדות לא רלונטיים

In [ ]:
parcel_x_process['CPBUniqueCancelProcessID'] = np.nan

parcel_x_process['ParcelType'] = 2 #סופי

parcel_x_process['LandDesignationPlan'] = parcel_x_process['LandDesignations']

parcel_x_process['ParcelUniqueID'] = parcel_x_process['ParcelUniqueID'] + 5000000

int_columns = ['ParcelUniqueID', 'ParcelType', 'ParcelNumber', 'BlockUniqueID', 'CreateProcessType', 'CPBUniqueCreateProcessID', 'CPBUniqueUpdateProcessID', 'CPBUniqueCancelProcessID']
parcel_x_process[int_columns] = parcel_x_process[int_columns].fillna(0)
parcel_x_process[int_columns] = parcel_x_process[int_columns].astype(int)

parcel_x_process['LegalArea'] = parcel_x_process['LegalArea'].fillna(0)
parcel_x_process['LegalArea'] = round(parcel_x_process['LegalArea']).astype(int)


parcel_x_process.drop_duplicates(subset = ['ParcelName'], keep = 'first', inplace=True)

parcel_x_process.drop(columns=['BlockName', 'ParcelName', 'ProcessName', 'LandDesignations'], inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  11.e Exporting results to product folder')

In [ ]:
#csv_dir  = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\parcels2D.csv'
json_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\parcels2D.geojson'


#parcel_x_process.drop(columns='geometry').to_csv(csv_dir ,encoding='cp1255', index=False)
gpd.GeoDataFrame(parcel_x_process, geometry = 'geometry', crs = 2039).to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Parcels2D-Save as Feature Class.bat"]);